<p align="center">
 <img src="http://www.di.uoa.gr/themes/corporate_lite/logo_en.png" title="Department of Informatics and Telecommunications - University of Athens"/> </p>

<br>

---

<h3 align="center" > 
  Bachelor Thesis
</h3>

<h1 align="center" > 
  Entity Resolution in Dissimilarity Spaces <br>
  Implementation notebook
</h1>

---

<h3 align="center"> 
 <b>Konstantinos Nikoletos</b>
</h3>

<h4 align="center"> 
 <b>Supervisor: Dr. Alex Delis</b>,  Professor NKUA
</h4>
<br>
<h4 align="center"> 
Athens
</h4>
<h4 align="center"> 
January 2021 - Ongoing
</h4>


---


|  <font size="5"> Contents</font> |
| :--   |
|**1. [Abstract](#Abstract)** |
|**2. [Introduction](#Introduction)**  |
&nbsp;&nbsp;&nbsp;**2.1. [   Entity resolution](#Entity-resolution)** |
&nbsp;&nbsp;&nbsp;**2.2. [   Dissimilatiry space](#Dissimilatiry-Space)** |
|**3. [ A dissimilarity-based space embedding methodology](#scrollTo=DcAYuFQjY2ni)** <br>
&nbsp;&nbsp;&nbsp;**3.1 [String Clustering and Prototype Selection](#3.1-String-Clustering-and-Prototype-Selection)** <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**3.1.1. [Edit distance metric](#Edit-distance-metric)** <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**3.1.2. [String clustering algorithm](#String-clustering-algorithm)** <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**3.1.3. [Algorithm complexity](#Algorithm-complexity)** <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**3.1.4. [Prototype selection](#Prototype-selection)** <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**3.1.5. [Algorithm-1: The String Clustering and Prototype Selection Algorithm](#Algorithm-1:-The-String-Clustering-and-Prototype-Selection-Algorithm)** <br>
&nbsp;&nbsp;&nbsp;**3.2 [The Vantage Space Embedding and the Chorus of Prototypes Transform Similarity Coefficient](#3.2-The-Vantage-Space-Embedding-and-the-Chorus-of-Prototypes-Transform-Similarity-Coefficient)&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;**  <br>
&nbsp;&nbsp;&nbsp;**3.3 [A Top-k List Approach for Similarity Searching in the Vantage Space](#3.3-A-Top-k-List-Approach-for-Similarity-Searching-in-the-Vantage-Space)**  |
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**3.3.1. [Abstract Algebra definitions](#Abstract-Algebra-definitions)** <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**3.3.2. [Hausdorff metric](#Hausdorff-metric)** <br>
&nbsp;&nbsp;&nbsp;**3.4 [Hashing of Partially Ranked Data for Efficient Similarity Search](#3.4-Hashing-of-Partially-Ranked-Data-for-Efficient-Similarity-Search)** |
|**4. [ Evaluation](#Evaluation)** |
|**5. [References](#References)**  |



# __Implementation__

## __0. Libraries__

In [3]:
import pandas as pd
import numpy as np
import collections

## __1. Prototype selection algorithm__

In [34]:
#####################################################################
# 1. Prototype selection algorithm                                  #
#####################################################################

'''
Clustering_Prototypes(S,k,d,r,C) 
The String Clustering and Prototype Selection Algorithm
is the main clustering method, that takes as input the intial strings S, 
the max number of clusters to be generated in k,
the maximum allowable distance of a string to join a cluster in var d
and returns the prototype for each cluster in array Prototype
'''
def Clustering_Prototypes(S,k,d,pairDictionary,verbose=True):
    
    # ----------------- Initialization phase ----------------- #
    i = 0
    j = 0
    C = np.empty([S_set.size], dtype=int)
    r = np.empty([2,k],dtype=object)

    Clusters = [ [] for l in range(0,k)]

    while i < S.size:     # String-clustering phase, for all strings
        while j < k :       # iteration through clusters, for all clusters
            if r[0][j] == None:      # case empty first representative for cluster j
                r[0][j] = S[i]   # init cluster representative with string i
                C[i] = j         # store in C that i-string belongs to cluster j
                Clusters[j].append(S[i])
                break
            elif r[1][j] == None and (EditDistance(S[i],r[0][j],pairDictionary) <= d):  # case empty second representative 
                r[1][j] = S[i]                                             # and ED of representative 1  smaller than i-th string 
                C[i] = j
                Clusters[j].append(S[i])
                break
            elif (r[0][j] != None and r[1][j] != None) and (EditDistance(S[i],r[0][j],pairDictionary) + EditDistance(S[i],r[1][j],pairDictionary)) <= d:
                C[i] = j
                Clusters[j].append(S[i])
                break
            else:
                j += 1
        i += 1
    
    # ----------------- Prototype selection phase ----------------- #
        
    Projections = np.empty([k],dtype=object)
    Prototypes = np.empty([k],dtype=int)
    sortedProjections = np.empty([k],dtype=object)
    j = 0

    if verbose:
        print("- - - - - - - - -")
        print("Cluster array:")
        print(C)
        print("- - - - - - - - -")
        print("Represantatives array:")
        print(r)
        print("- - - - - - - - -")  
        print("Clusters:")
        print(Clusters)
        print("- - - - - - - - -")  

    
    while j < k and len(Clusters[j])>0:
        print("\n\n\n****** Prototype selection phase *********") 
        Projections[j] = Approximated_Projection_Distances_ofCluster(r[1][j], r[0][j], j, Clusters[j],pairDictionary)
        # sortedProjections[j] = np.sort(np.array(Projections[j]),kind = 'quicksort' ) 
        
        print("\n"+str(j)+"-Projections:")
        print(Projections[j])
        
        sortedProjections[j] = {k: v for k, v in sorted(Projections[j].items(), key=lambda item: item[1])}

        print(str(j)+"-sortedProjections:")
        print(sortedProjections[j])
        
        Prototypes[j] = Median(sortedProjections[j])
        
        print(".............")
        print(str(j)+"-Prototypes:")        
        print(Prototypes[j])
        
        j += 1
        print("\n****** END *********\n")

    return Prototypes

def EditDistance(str1,str2,pairDictionary,verbose=False):
    if verbose:
        if str1 == None:
            print("1")
        elif str2 == None:
            print("2")
        print("-> "+str(str1))
        print("--> "+str(str2))
        print(str(editdistance.eval(S_set[str1],S_set[str2])))
    
    
    # NOTE: Duplicates inside the dictionary     

    if ((str1,str2) or (str2,str1))  in pairDictionary.keys():
        return pairDictionary[(str1,str2)]
    else:
        distance = editdistance.eval(S_set[str1],S_set[str2])
        pairDictionary[(str2,str1)] = pairDictionary[(str1,str2)] = distance
        return distance


def Approximated_Projection_Distances_ofCluster(right_rep, left_rep, cluster_id, clusterSet,pairDictionary):

    distances_vector = dict()
    rep_distance     = EditDistance(right_rep,left_rep)

    for str_inCluster in range(0,len(clusterSet)): 

      right_rep_distance = EditDistance(right_rep,clusterSet[str_inCluster])
      left_rep_distance  = EditDistance(left_rep,clusterSet[str_inCluster])
      
      distance = (right_rep_distance**2-rep_distance**2-left_rep_distance**2 ) / (2*rep_distance)
      distances_vector[clusterSet[str_inCluster]] = distance

    return distances_vector

def Median(distances):    
    '''
    Returns the median value of a vector
    '''
    keys = list(distances.keys())
    median_position = int(len(keys)/2)
    median_value = keys[median_position]

    return median_value

In [ ]:
!pip install editdistance

In [ ]:
import editdistance

print("Edit distance: "+str(editdistance.eval('banana', 'bahama')))

Edit distance: 2


In [ ]:
pairDictionary = dict()
input_strings = list(strObjects)
k = 7 # max_number_of_clusters
d = 120
S_set = np.array(input_strings,dtype=object)
S_index = np.arange(0,len(input_strings),1)
# S_index_list = list(S_index) 


# print(S_set)

print("\n-----------------\nString positions are:")
print(S_index)
print("-----------------\n")

Prototypes = Clustering_Prototypes(S_index,k,d)

print("\n-----------------\n NumofPrototypes: "+str(k)+" || Prototypes are: ")
print(Prototypes)
print("-----------------")


-----------------
String positions are:
[   0    1    2 ... 2291 2292 2293]
-----------------

- - - - - - - - -
Cluster array:
[     0      0      0 ... 672980 945741 672979]
- - - - - - - - -
Represantatives array:
[[0 11 13 25 28 31 40]
 [1 12 14 26 29 32 41]]
- - - - - - - - -
Clusters:
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [11, 12], [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24], [25, 26, 27], [28, 29, 30], [31, 32, 33, 34, 35, 36, 37, 38, 39], [40, 41]]
- - - - - - - - -



****** Prototype selection phase *********

0-Projections:
{0: 0.0, 1: -56.0, 2: -28.0, 3: -21.0, 4: -11.107142857142858, 5: -16.044642857142858, 6: -24.142857142857142, 7: -11.544642857142858, 8: -23.3125, 9: -26.285714285714285, 10: -25.964285714285715}
0-sortedProjections:
{1: -56.0, 2: -28.0, 9: -26.285714285714285, 10: -25.964285714285715, 6: -24.142857142857142, 8: -23.3125, 3: -21.0, 5: -16.044642857142858, 7: -11.544642857142858, 4: -11.107142857142858, 0: 0.0}
.............
0-Prototypes:
8

****** E

In [ ]:
from scipy import stats 

In [ ]:
x1 = [12, 2, 1, 12, 2]
x2 = [1, 4, 7, 1, 0]

stats.rankdata([5,9,10,2], method='dense')

array([2, 3, 4, 1])

## __2. Embeddings based on the Vantage objects__




In [ ]:
#####################################################################
#       2. Embeddings based on the Vantage objects                  #
#####################################################################

'''
CreateVantageEmbeddings(S,VantageObjects): Main function for creating the string embeddings based on the Vantage Objects
'''
def CreateVantageEmbeddings(S,VantageObjects):
    
    # ------- Distance computing ------- #     
    vectors = []
    for s in range(0,S.size):
        string_embedding = []
        for p in range(0,VantageObjects.size): 
            if VantageObjects[p] != None:
                string_embedding.append(DistanceMetric(s,p,S,VantageObjects))
            
        # --- Ranking representation ---- #
        ranked_string_embedding = stats.rankdata(string_embedding, method='dense')
        
        # ------- Vectors dataset ------- #
        vectors.append(ranked_string_embedding)
    
    return np.array(vectors)
    

'''
DistanceMetric(s,p,S,Prototypes): Implementation of equation (5)
'''
def DistanceMetric(s,p,S,VantageObjects):
    
    max_distance = None
    
    for pp in range(0,VantageObjects.size):
        if VantageObjects[pp] != None:
            string_distance = EditDistance(S[s],VantageObjects[pp])    # Edit distance String-i -> Vantage Object
            VO_distance     = EditDistance(VantageObjects[p],VantageObjects[pp])    # Edit distance Vantage Object-j -> Vantage Object-i

            abs_diff = abs(string_distance-VO_distance)

            # --- Max distance diff --- #        
            if max_distance == None:
                max_distance = abs_diff
            elif abs_diff > max_distance:
                max_distance = abs_diff
            
    return max_distance

def dropNone(array):
    array = list(filter(None, list(array)))
    return np.array(array)

def topKPrototypes():
    return

In [ ]:
Embeddings = CreateVantageEmbeddings(S_index,Prototypes)

In [ ]:
Embeddings[1]

array([2, 6, 1, 3, 4, 4, 5])

In [ ]:
print(Embeddings[1])
print(Embeddings[3])

[2 6 1 3 4 4 5]
[4 6 4 3 1 2 5]


## __3. Metrics and Similarity functions__

In [ ]:
tau, p_value = stats.kendalltau(Embeddings[1], Embeddings[2])
print(tau,p_value)

0.44999999999999996 0.16682603139313168


In [ ]:
from scipy.spatial.distance import directed_hausdorff
from scipy.spatial.distance import hamming

print(Embeddings[1].reshape(1, -1))
print(Embeddings[2].reshape(1, -1))

haus,_,_ = directed_hausdorff(Embeddings[1].reshape(1, -1), Embeddings[2].reshape(1, -1))
print(haus)

[[2 6 1 3 4 4 5]]
[[3 5 2 4 3 1 6]]
3.872983346207417


In [ ]:
from scipy.spatial.distance import hamming

hamming(list(Embeddings[1]),list(Embeddings[3]))

0.5714285714285714

In [ ]:
#####################################################################
#                 3. Similarity function                            # 
#####################################################################




## __4. Hashing__

In [ ]:
#####################################################################
#                        4. Hashing                                 # 
#####################################################################

def WTA(vectors,K,inputDim):
  '''
    Winner Take All hash - Yagnik
    .............................

    m: number of permutations
    K: window size
  '''
  C = np.zeros([vectors.size], dtype=int)
  theta = np.random.permutation(inputDim)
  for v in vectors:
    X_new = permuted(v)
    index_max = max(range(len(values)), key=values.__getitem__)
    c_i = index_max

    j=0
    for j in range(0,K-1):
      if X_new[j] > X_new[c_i]:
        c_i = j

    C[i] = c_i

  return C

def permuted(vector,permutation):
  permuted_vector = [vector[x] for x in permutation]
  return permuted_vector 

## __Final model__









In [33]:
class RankedWTAHash:

  def __init__(self, max_numberOf_clusters, max_editDistance ):
      '''
        Constructor
      '''
      self.question = question
      self.max_numberOf_clusters = max_numberOf_clusters
      self.pairDictionary = dict()
      self.max_editDistance = max_editDistance
      self.windowSize = windowSize
      
      # self.inputSize = 
  
  def fit(self, X, y):
    """
      Fit the classifier from the training dataset.
      Parameters
      ----------
      X : Training data.
      y : Target values.
      Returns
      -------
      self : The fitted classifier.
    """
    input_strings = list(X)
    S_set = np.array(input_strings,dtype=object)
    S_index = np.arange(0,len(input_strings),1)
    # S_index_list = list(S_index) 
    self.X = S_index

    # print(S_set)

    print("\n-----------------\nString positions are:")
    print(S_index)
    print("-----------------\n")

    self.prototypeArray = Clustering_Prototypes(S_index,self.max_numberOf_clusters, self.max_editDistance, self.pairDictionary)
    self.embeddingDim   = self.prototypeArray.size
    print("\n-----------------\n NumofPrototypes: "+str(k)+" || Prototypes are: ")
    print(Prototypes)
    print("-----------------")

    self.Embeddings = CreateVantageEmbeddings(self.X,self.prototypeArray)


    self.HashedClusters = self.WTA(self.Embeddings,self.windowSize,self.embeddingDim)

    return self


  
  # def predict(self, X):
  # """
  #   Predict the class labels for the provided data.
  #   Parameters
  #   ----------
  #   X : Test samples.
  #   Returns
  #   -------
  #   y : Class labels for each data sample.
  # """
  
  
  # def predict_proba(self, X):
  # """
  #   Return probability estimates for the test data X.
  #   Parameters
  #   ----------
  #   X : Test samples.
    
  #   Returns
  #   -------
  #   p : The class probabilities of the input samples.
  # """


  # # def evaluate():

---
---

# __Evaluation__

In [1]:
# Opening data file
import io
from google.colab import drive

drive.mount('/content/drive',force_remount=True)

## __CoRA__

### Load from Drive

In [6]:
fpcites = r"/content/drive/My Drive/ERinDS/cora_cites.csv"
fppaper = r"/content/drive/My Drive/ERinDS/cora_paper.csv"
fpcontent = r"/content/drive/My Drive/ERinDS/cora_content.csv"

cites = pd.read_csv(fpcites,sep=';')
paper = pd.read_csv(fppaper,sep=';')
content = pd.read_csv(fpcontent,sep=';')

Mounted at /content/drive


### Overview

In [7]:
cites

,cited_paper_id,citing_paper_id
0,35,887
1,35,1033
2,35,1688
3,35,1956
4,35,8865
...,...,...
5424,853116,19621
5425,853116,853155
5426,853118,1140289
5427,853155,853118


In [8]:
paper

,paper_id,class_label
0,35,Genetic_Algorithms
1,40,Genetic_Algorithms
2,114,Reinforcement_Learning
3,117,Reinforcement_Learning
4,128,Reinforcement_Learning
...,...,...
2703,1154500,Case_Based
2704,1154520,Neural_Networks
2705,1154524,Rule_Learning
2706,1154525,Rule_Learning


In [18]:
content

,paper_id,word_cited_id
0,35,word100
1,35,word1152
2,35,word1175
3,35,word1228
4,35,word1248
...,...,...
49211,1155073,word75
49212,1155073,word759
49213,1155073,word789
49214,1155073,word815


### Train-Test-Validation datasets



In [13]:
class_labels = np.unique(paper.class_label.to_numpy())
print(class_labels)
print("Number of classes: "+str(len(class_labels)))

['Case_Based' 'Genetic_Algorithms' 'Neural_Networks'
 'Probabilistic_Methods' 'Reinforcement_Learning' 'Rule_Learning' 'Theory']
Number of classes: 7


## __DBLP/ACM__

In [24]:
acmfp = r"/content/drive/My Drive/ERinDS/ACM.csv"
dblpfp = r"/content/drive/My Drive/ERinDS/DBLP2.csv"
acm_dblp_mapping_fp = r"/content/drive/My Drive/ERinDS/DBLP-ACM_perfectMapping.csv"

acm = pd.read_csv(acmfp)
dblp = pd.read_csv(dblpfp, encoding='latin-1')
perfect_mapping = pd.read_csv(acm_dblp_mapping_fp)

### Overview

In [ ]:
acm

,id,title,authors,venue,year
0,304586,The WASA2 object-oriented workflow management ...,"Gottfried Vossen, Mathias Weske",International Conference on Management of Data,1999
1,304587,A user-centered interface for querying distrib...,"Isabel F. Cruz, Kimberly M. James",International Conference on Management of Data,1999
2,304589,"World Wide Database-integrating the Web, CORBA...","Athman Bouguettaya, Boualem Benatallah, Lily H...",International Conference on Management of Data,1999
3,304590,XML-based information mediation with MIX,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228...",International Conference on Management of Data,1999
4,304582,The CCUBE constraint object-oriented database ...,"Alexander Brodsky, Victor E. Segal, Jia Chen, ...",International Conference on Management of Data,1999
...,...,...,...,...,...
2289,672977,Dual-Buffering Strategies in Object Bases,"Alfons Kemper, Donald Kossmann",Very Large Data Bases,1994
2290,950482,Guest editorial,"Philip A. Bernstein, Yannis Ioannidis, Raghu R...",The VLDB Journal &mdash; The International Jou...,2003
2291,672980,GraphDB: Modeling and Querying Graphs in Datab...,Ralf Hartmut G&#252;ting,Very Large Data Bases,1994
2292,945741,Review of The data warehouse toolkit: the comp...,Alexander A. Anisimov,ACM SIGMOD Record,2003


In [22]:
dblp

,id,title,authors,venue,year
0,journals/sigmod/Mackay99,Semantic Integration of Environmental Models f...,D. Scott Mackay,SIGMOD Record,1999
1,conf/vldb/PoosalaI96,Estimation of Query-Result Distribution and it...,"Viswanath Poosala, Yannis E. Ioannidis",VLDB,1996
2,conf/vldb/PalpanasSCP02,Incremental Maintenance for Non-Distributive A...,"Themistoklis Palpanas, Richard Sidle, Hamid Pi...",VLDB,2002
3,conf/vldb/GardarinGT96,Cost-based Selection of Path Expression Proces...,"Zhao-Hui Tang, Georges Gardarin, Jean-Robert G...",VLDB,1996
4,conf/vldb/HoelS95,Benchmarking Spatial Join Operations with Spat...,"Erik G. Hoel, Hanan Samet",VLDB,1995
...,...,...,...,...,...
2611,journals/tods/KarpSP03,A simple algorithm for finding frequent elemen...,"Scott Shenker, Christos H. Papadimitriou, Rich...",ACM Trans. Database Syst.,2003
2612,conf/vldb/LimWV03,SASH: A Self-Adaptive Histogram Set for Dynami...,"Lipyeow Lim, Min Wang, Jeffrey Scott Vitter",VLDB,2003
2613,journals/tods/ChakrabartiKMP02,Locally adaptive dimensionality reduction for ...,"Kaushik Chakrabarti, Eamonn J. Keogh, Michael ...",ACM Trans. Database Syst.,2002
2614,journals/sigmod/Snodgrass01,Chair's Message,Richard T. Snodgrass,SIGMOD Record,2001


In [25]:
perfect_mapping

,idDBLP,idACM
0,conf/sigmod/SlivinskasJS01,375678
1,conf/sigmod/ChaudhuriDN01,375694
2,conf/sigmod/RinfretOO01,375669
3,conf/sigmod/BreunigKKS01,375672
4,conf/sigmod/JagadishJOT01,375687
...,...,...
2219,journals/sigmod/Scholl01,604275
2220,journals/sigmod/Rosneblatt94,190649
2221,journals/sigmod/Winslett02b,601871
2222,journals/sigmod/Labrinidis01,604283


In [30]:
acm.loc[acm['id'] == 375678]

,id,title,authors,venue,year
301,375678,Adaptable query optimization and evaluation in...,"Giedrius Slivinskas, Christian S. Jensen, Rich...",International Conference on Management of Data,2001


In [32]:
dblp.loc[dblp['id'] == 'conf/sigmod/SlivinskasJS01']

,id,title,authors,venue,year
143,conf/sigmod/SlivinskasJS01,Adaptable Query Optimization and Evaluation in...,"Christian S. Jensen, Richard T. Snodgrass, Gie...",SIGMOD Conference,2001


### Preprocess

### Dataset split

### Model evaluation

---

# References

[1]   [The dissimilarity representation for pattern recognition, a tutorial
Robert P.W. Duin and Elzbieta Pekalska Delft University of Technology, The Netherlands School of Computer Science, University of Manchester, United Kingdom](http://homepage.tudelft.nl/a9p19/presentations/DisRep_Tutorial_doc.pdf)